In [1]:
import os
os.chdir("../")

In [2]:
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter


c:\Users\ibtih\miniconda3\envs\mchatbot\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
#Extract Data From the PDF File
def load_pdf_file(data):
    loader= DirectoryLoader(data,
    glob="*.pdf",
    loader_cls=PyPDFLoader)
    documents=loader.load()
    return documents

In [4]:
extracted_data=load_pdf_file(data="C:\End-to-end-medical-chatbot\Data")

In [5]:
#Split the Data into Text Chunks
def text_split(extracted_data):
    text_splitter=RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=20)
    text_chunks=text_splitter.split_documents(extracted_data)
    return text_chunks

In [6]:
text_chunks=text_split(extracted_data)
print("Length of Text Chunks: ", len(text_chunks))

Length of Text Chunks:  5859


In [7]:
from langchain_community.embeddings import HuggingFaceEmbeddings

In [ ]:
def download_hugging_face_embeddings():
    embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    return embeddings

In [9]:
embeddings = download_hugging_face_embeddings()

C:\Users\ibtih\AppData\Local\Temp\ipykernel_248188\4238859041.py:2: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")


In [42]:
pip install pinecone


In [31]:
from dotenv import load_dotenv
load_dotenv()

False

In [32]:
PINECONE_API_KEY=os.environ.get('PINECONE_API_KEY')
GROK_API_KEY=os.environ.get('GROK_API_KEY')

In [14]:
from pinecone.grpc import PineconeGRPC as Pinecone
from pinecone import ServerlessSpec
import os
pc = Pinecone(api_key=PINECONE_API_KEY)
index_name = "mchatbot"

pc.create_index(
        name=index_name,
        dimension=384,
        metric="cosine",
        spec=ServerlessSpec(
            cloud="aws",
        region="us-east-1"
        )
       
    )

PineconeApiException: (409)
Reason: Conflict
HTTP response headers: HTTPHeaderDict({'content-type': 'text/plain; charset=utf-8', 'access-control-allow-origin': '*', 'vary': 'origin,access-control-request-method,access-control-request-headers', 'access-control-expose-headers': '*', 'x-pinecone-api-version': '2025-04', 'x-cloud-trace-context': 'b6ca2cc29ae6d202201b73085091afeb', 'date': 'Wed, 17 Sep 2025 18:48:16 GMT', 'server': 'Google Frontend', 'Content-Length': '85', 'Via': '1.1 google', 'Alt-Svc': 'h3=":443"; ma=2592000,h3-29=":443"; ma=2592000'})
HTTP response body: {"error":{"code":"ALREADY_EXISTS","message":"Resource  already exists"},"status":409}


In [33]:
import os

os.environ["PINECONE_API_KEY"] = PINECONE_API_KEY
os.environ["GROK_API_KEY"] = GROK_API_KEY

In [16]:
# Embed each chunk and upsert the embeddings into your Pinecone index.
from langchain_pinecone import PineconeVectorStore

docsearch = PineconeVectorStore.from_documents(
    documents=text_chunks,
    index_name=index_name,
    embedding=embeddings
)

KeyboardInterrupt: 

In [17]:
# Load Existing index
from langchain_pinecone import PineconeVectorStore

# Load existing Pinecone index
docsearch = PineconeVectorStore.from_existing_index(
    index_name=index_name,
    embedding=embeddings
)

In [18]:
retriever = docsearch.as_retriever(
    search_type="similarity",
    search_kwargs={"k": 3}
)

In [38]:
from langchain_core.runnables import Runnable
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate
import os
from dotenv import load_dotenv
import requests
import json

# Load environment variables
load_dotenv()
GROK_API_KEY = os.environ.get('GROK_API_KEY')  # This is actually your Groq API key

# Custom GroqChatRunnable class to integrate with LangChain
class GroqChatRunnable(Runnable):
    def __init__(self, api_key, model="llama-3.1-8b-instant", temperature=0.7):
        self.api_key = api_key
        self.model = model
        self.temperature = temperature

    def invoke(self, input_data, config=None):
        # Handle LangChain's input format
        if isinstance(input_data, dict):
            # Extract system prompt and user input
            messages = [
                {"role": "system", "content": input_data.get("system_prompt", "")},
                {"role": "user", "content": input_data.get("input", "")}
            ]
        else:
            # Fallback for other input formats
            messages = [{"role": "user", "content": str(input_data)}]

        # Make API call to Groq
        url = "https://api.groq.com/openai/v1/chat/completions"
        headers = {
            "Authorization": f"Bearer {self.api_key}",
            "Content-Type": "application/json"
        }
        data = {
            "messages": messages,
            "model": self.model,
            "temperature": self.temperature,
            "stream": False
        }
        response = requests.post(url, headers=headers, json=data)
        if response.status_code == 200:
            # Return the raw content string to match LangChain's expected output
            return response.json()["choices"][0]["message"]["content"]
        else:
            raise Exception(f"API request failed with status {response.status_code}: {response.text}")

    def bind_tools(self, *args, **kwargs):
        return self  # Simplified for this use case

# Initialize Groq model as a Runnable
llm = GroqChatRunnable(api_key=GROK_API_KEY, model="llama-3.1-8b-instant", temperature=0.7)

# Define system prompt
system_prompt = (
    "You are a medical assistant for question-answering tasks. "
    "Use the provided context from medical documents to answer questions accurately. "
    "If you don't know the answer, admit it and keep the response to three sentences maximum."
    "\n\n"
    "{context}"
)

# Create the prompt template
prompt = ChatPromptTemplate.from_messages([
    ("system", system_prompt),
    ("human", "{input}")
])

# Create the combine documents chain
question_answer_chain = create_stuff_documents_chain(llm, prompt)

# Create the retrieval chain
rag_chain = create_retrieval_chain(retriever, question_answer_chain)

In [40]:
response = rag_chain.invoke({"input": "What is Acne?"})
print(response["answer"])

Based on the provided messages, it appears that the HumanMessage is asking a question about what Acne is. 

To answer this question accurately, I will refer to the provided context from the GALE ENCYCLOPEDIA OF MEDICINE. 

Here's my response:

Acne is a skin disorder in which the sebaceous glands become inflamed. It is commonly referred to as acne vulgaris. Acne affects the skin and can appear on the face and other areas of the body.


In [41]:
response = rag_chain.invoke({"input": "What is the meaning of cha7ba?"})
print(response["answer"])

Based on the provided messages, it seems like there are multiple messages, but only one of them is a HumanMessage asking a question.

Since the HumanMessage is asking a question, I'll try to find the answer from the context provided. 

However, I couldn't find any information related to "cha7ba" in the context provided. I'll respond based on the information given in the context.

Response: I couldn't find any information related to "cha7ba" in the medical documents provided.
